In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# MyRealty
from ConcreteScrapers.MyRealty.MyRealtyScrapingPipeline import MyRealtyScrapingPipeline

# Storage
from ConcreteStorages import CSVStorage
from Services import ImageLoader

In [8]:
storage = CSVStorage("apartments.csv")
image_loader = ImageLoader(storage)

In [9]:
scraper_pipeline = MyRealtyScrapingPipeline(
    "https://myrealty.am/en/apartments-for-sale/7784", 
    storage,
    image_loader
)

TypeError: __init__() missing 1 required positional argument: 'image_loader'

# GeoService

In [1]:
from GeoFeaturesExtractor import GeoFeaturesExtractor
geoService = GeoFeaturesExtractor(["supermarket", "restaurant", "store", "hospital", "subway_station", "school", "beauty_salon"], 1000)
lat, lng = geoService.location("Yerevan/Erebuni/Khorenatsi Street (ER.)")
amenities = geoService.find_nearby_places(f"{lat},{lng}")
print(amenities)

[Amenity: store, Latitude: 40.1503659, Longitude: 44.5231357, Amenity: restaurant, Latitude: 40.1416798, Longitude: 44.5222183, Amenity: school, Latitude: 40.14604459999999, Longitude: 44.5233545, Amenity: school, Latitude: 40.140908, Longitude: 44.5203911, Amenity: beauty_salon, Latitude: 40.1456645, Longitude: 44.52097089999999, Amenity: store, Latitude: 40.1421264, Longitude: 44.5211657, Amenity: supermarket, Latitude: 40.1421264, Longitude: 44.5211657, Amenity: store, Latitude: 40.1423248, Longitude: 44.5224935]


In [74]:
from ConcreteScrapers.Bars.BarsApartmentScraper import BarsApartmentScraper
scraper = BarsApartmentScraper("https://bars.am/en/properties/standard/apartment/53896")
scraper.scrape()
scraper.values()

{'id': '53896',
 'price': 75000,
 'facilities': ['Building security',
  'Heating system',
  'Open balcony',
  'Washing machine',
  'Kitchen stove',
  'Fridge',
  'Built-in furniture',
  'Kitchen furniture',
  'Transport availability'],
 'location': 'Yerevan/Erebuni/Khorenatsi Street (ER.)',
 'area': 78.0,
 'rooms': 3,
 'floor': 2,
 'storeys': 4,
 'bedrooms': 2,
 'bathrooms': 1,
 'ceiling_height': 3.2,
 'building_type': 'Stalinka c/o',
 'condition': 'Old major overhaul'}